In [9]:
import json
from enum import Enum
from typing import List
from pydantic import BaseModel, Field
import ollama
import re
from typing import Dict, Any

In [10]:

class TicketCategory(str, Enum):
    ORDER_ISSUE = "order_issue"
    ACCOUNT_ACCESS = "account_access"
    PRODUCT_INQUIRY = "product_inquiry"
    TECHNICAL_SUPPORT = "technical_support"
    BILLING = "billing"
    OTHER = "other"

class CustomerSentiment(str, Enum):
    ANGRY = "angry"
    FRUSTRATED = "frustrated"
    NEUTRAL = "neutral"
    SATISFIED = "satisfied"

class TicketUrgency(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

In [11]:
class TicketClassification(BaseModel):
    category: TicketCategory
    urgency: TicketUrgency
    sentiment: CustomerSentiment
    confidence: float = Field(ge=0, le=1, description="Confidence score for the classification")
    key_information: List[str] = Field(description="List of key points extracted from the ticket")
    suggested_action: str = Field(description="Brief suggestion for handling the ticket")


In [12]:
def preprocess_json(json_string: str) -> str:
    # Remove any text before the first '{' and after the last '}'
    json_string = re.sub(r'^[^{]*', '', json_string)
    json_string = re.sub(r'[^}]*$', '', json_string)

    # Replace single quotes with double quotes
    json_string = json_string.replace("'", '"')

    # Remove any trailing commas before closing braces or brackets
    json_string = re.sub(r',\s*([\]}])', r'\1', json_string)

    return json_string

In [13]:
def parse_json_safely(json_string: str) -> Dict[str, Any]:
    try:
        return json.loads(json_string)
    except json.JSONDecodeError:
        # If standard JSON parsing fails, try a more lenient approach
        import ast
        return ast.literal_eval(json_string)

In [14]:
def classify_ticket(ticket_text: str, max_retries: int = 3) -> TicketClassification:
    prompt = f"""
    Analyze the following customer support ticket and provide a classification in JSON format:

    Ticket: {ticket_text}

    Respond with a JSON object containing ONLY the following fields:
    {{
        "category": "order_issue" | "account_access" | "product_inquiry" | "technical_support" | "billing" | "other",
        "urgency": "low" | "medium" | "high" | "critical",
        "sentiment": "angry" | "frustrated" | "neutral" | "satisfied",
        "confidence": <float between 0 and 1>,
        "key_information": [<list of key points as strings>],
        "suggested_action": "<brief suggestion for handling the ticket>"
    }}

    Ensure the response is a valid JSON object. Do not include any additional text or explanations.
    """

    for attempt in range(max_retries):
        try:
            response = ollama.chat(model='llama3:latest', messages=[
                {'role': 'system', 'content': 'You are a helpful assistant that analyzes customer support tickets.'},
                {'role': 'user', 'content': prompt}
            ])

            preprocessed_content = preprocess_json(response['message']['content'])
            classification_dict = parse_json_safely(preprocessed_content)
            return TicketClassification(**classification_dict)
        except (json.JSONDecodeError, ValueError, KeyError) as e:
            if attempt == max_retries - 1:
                raise ValueError(f"Failed to parse response after {max_retries} attempts: {str(e)}")
            continue

    raise ValueError("Unexpected error in classification process")

In [17]:
ticket1 = """
I ordered a laptop from your store last week (Order #12345), but I received a tablet instead. 
This is unacceptable! I need the laptop for work urgently. Please resolve this immediately or I'll have to dispute the charge.
"""

ticket2 = """
Hello, I'm having trouble logging into my account. I've tried resetting my password, but I'm not receiving the reset email. 
Can you please help me regain access to my account? I've been a loyal customer for years and have several pending orders.
"""

In [18]:
classification = classify_ticket(ticket1)

In [19]:
classification

TicketClassification(category=<TicketCategory.ORDER_ISSUE: 'order_issue'>, urgency=<TicketUrgency.HIGH: 'high'>, sentiment=<CustomerSentiment.ANGRY: 'angry'>, confidence=0.9, key_information=['Order #12345', 'Received tablet instead of laptop', 'Urgent need for laptop'], suggested_action='Resolve the order discrepancy and expedite shipping or offer a replacement')